# How can the Euclidean distance be calculated with NumPy?
https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy


IMPORTS

In [583]:
import pandas as pd 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.spatial import distance
import numpy as np
pd.set_option('display.max_columns', None)

## NBA PLAYER DISTANCE 
- TEST 4 - DF OF VALUES + NORMALIZED + CORRECTED VALUES
- VERY GOOD !!! 

In [584]:
pd.set_option('display.max_columns', None)
from nba_api.stats.endpoints import leaguedashplayerbiostats
import pandas as pd 

output = leaguedashplayerbiostats.LeagueDashPlayerBioStats(league_id='00', per_mode_simple='PerGame', season='2020-21', season_type_all_star='Regular Season', college_nullable='', conference_nullable='', country_nullable='', date_from_nullable='', date_to_nullable='', division_simple_nullable='', draft_pick_nullable='', draft_year_nullable='', game_scope_simple_nullable='', game_segment_nullable='', height_nullable='', last_n_games_nullable='', location_nullable='', month_nullable='', opponent_team_id_nullable='', outcome_nullable='', po_round_nullable='', period_nullable='', player_experience_nullable='', player_position_abbreviation_nullable='', season_segment_nullable='', shot_clock_range_nullable='', starter_bench_nullable='', team_id_nullable='', vs_conference_nullable='', vs_division_nullable='', weight_nullable='', proxy=None, headers=None, timeout=30, get_request=True).get_data_frames()

z = pd.DataFrame(output[-1])
z[["PLAYER_WEIGHT"]] = z[["PLAYER_WEIGHT"]].apply(pd.to_numeric)
z.sort_values(by=('PTS'), ascending=False).head(2)

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GP,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT
189,201935,James Harden,1610612745,HOU,31.0,6-5,77,220,Arizona State,USA,2009,1,3,4,33.0,4.8,10.8,-10.7,0.021,0.107,0.332,0.672,0.478
384,201939,Stephen Curry,1610612744,GSW,32.0,6-3,75,185,Davidson,USA,2009,1,7,7,32.0,5.3,6.4,-4.4,0.015,0.123,0.331,0.645,0.349


In [589]:
i = 0
distarr = []
mindst = 100 
mindstID = 0

def reorder_columns(columns, first_cols=[], last_cols=[], drop_cols=[]):
    columns = list(set(columns) - set(first_cols))
    columns = list(set(columns) - set(drop_cols))
    columns = list(set(columns) - set(last_cols))
    new_order = first_cols + columns + last_cols
    return new_order

my_list = z.columns.tolist()
reordered_cols = reorder_columns(my_list, first_cols=['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_HEIGHT','COLLEGE','COUNTRY', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER'], last_cols=['AGE', 'PLAYER_HEIGHT_INCHES', 'PLAYER_WEIGHT','GP', 'PTS','REB', 'AST', 'NET_RATING', 'OREB_PCT', 'DREB_PCT', 'USG_PCT', 'TS_PCT','AST_PCT'], drop_cols=[])
z1 = z[reordered_cols]

# NORMALIZZAZIONE E FATTORI DI CORREZIONE 
features_to_normalize = ['AGE','PLAYER_HEIGHT_INCHES', 'PLAYER_WEIGHT','GP','PTS', 'REB', 'AST', 'NET_RATING', 'OREB_PCT', 'DREB_PCT', 'USG_PCT','TS_PCT', 'AST_PCT']
z1[features_to_normalize] = z1[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))
z1["GP"]=z1["GP"]/10
z1["PTS"]=z1["PTS"]*2
z1["NET_RATING"]=z1["NET_RATING"]*3
z1["AST_PCT"]=z1["AST_PCT"]*3
z1["USG_PCT"]=z1["USG_PCT"]*3
z1["PLAYER_HEIGHT_INCHES"]=z1["PLAYER_HEIGHT_INCHES"]/5
z1["PLAYER_WEIGHT"]=z1["PLAYER_WEIGHT"]/5

# z["PLAYER_WEIGHTn"] = z["PLAYER_WEIGHT"] / z["PLAYER_WEIGHT"].max()    ( per normalizzare una sola colonna usa questo )

targetplayer = z1[(z1.PLAYER_NAME=='Trae Young')].to_numpy()[0][10:]   # 0 for first row (data), 10 for first 10 columns to remove 

for i, row in z1.iterrows():
    testarray= row.to_numpy()
    testarray=testarray[10:] #10 first columns to exclude from distance calculation 
    z1.at[i, 'EuclDist']  = distance.euclidean(targetplayer, testarray)  # euclidean distance

z1.sort_values(by=('EuclDist'), ascending=True).head(10)

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,PLAYER_HEIGHT,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT,EuclDist
410,1629027,Trae Young,1610612737,ATL,6-1,Oklahoma,USA,2018,1,5,0.176471,0.031579,0.026490,0.085714,1.733333,0.292857,0.764706,2.258499,0.138138,0.148649,2.028,0.408000,1.272,0.000000
173,1629630,Ja Morant,1610612763,MEM,6-3,Murray State,USA,2019,1,2,0.117647,0.052632,0.018543,0.028571,1.593939,0.164286,0.529412,2.187323,0.066066,0.137387,2.058,0.415333,1.119,0.365112
35,1627742,Brandon Ingram,1610612740,NOP,6-7,Duke,USA,2016,1,2,0.235294,0.094737,0.039735,0.085714,1.515152,0.457143,0.478992,2.155453,0.048048,0.385135,1.812,0.378000,0.888,0.658424
275,1629029,Luka Doncic,1610612742,DAL,6-7,None,Slovenia,2018,1,3,0.117647,0.094737,0.092715,0.071429,1.563636,0.642857,0.605042,2.178824,0.078078,0.592342,2.232,0.362667,1.266,0.662839
262,202681,Kyrie Irving,1610612751,BKN,6-2,Duke,Australia,2011,1,1,0.529412,0.042105,0.046358,0.085714,1.642424,0.378571,0.512605,2.315864,0.063063,0.252252,1.758,0.420000,0.867,0.678947
189,201935,James Harden,1610612745,HOU,6-5,Arizona State,USA,2009,1,3,0.705882,0.073684,0.079470,0.042857,2.000000,0.342857,0.907563,2.010977,0.063063,0.240991,1.992,0.448000,1.434,0.696622
237,202695,Kawhi Leonard,1610612746,LAC,6-7,San Diego State,USA,2011,1,15,0.588235,0.094737,0.086093,0.071429,1.412121,0.414286,0.546218,2.257436,0.150150,0.277027,1.794,0.349333,0.936,0.728474
384,201939,Stephen Curry,1610612744,GSW,6-3,Davidson,USA,2009,1,7,0.764706,0.052632,0.033113,0.085714,1.939394,0.378571,0.537815,2.077904,0.045045,0.277027,1.986,0.430000,1.047,0.747282
75,203081,Damian Lillard,1610612757,POR,6-2,Weber State,USA,2012,1,6,0.647059,0.042105,0.046358,0.085714,1.575758,0.314286,0.554622,2.094901,0.021021,0.261261,1.716,0.392667,0.873,0.775452
91,1628368,De'Aaron Fox,1610612758,SAC,6-3,Kentucky,USA,2017,1,5,0.235294,0.052632,0.033113,0.085714,1.248485,0.221429,0.529412,2.063031,0.066066,0.155405,1.650,0.342000,0.936,0.776579


## ----------------- TMP --------------


In [586]:
# TEST 1 - ONE DISTANCE 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.spatial import distance
import numpy as np

a = (1, 2, 3)
b = (4, 5, 6)
dst = distance.euclidean(a, b)
a
b
dst

(1, 2, 3)

(4, 5, 6)

5.196152422706632

In [587]:
# TEST 2 -  ARRAY OF VALUES : FIND CLOSER ARRAYS

rdnm = np.random.randint(0,20,(1000,5))
cfrf = np.random.randint(0,20,(1,5))
distarr = []
mindst = 100 
mindstID = 0

for i in range(len(rdnm)):
    thisdist = distance.euclidean(cfrf, rdnm[i])  # euclidean distance
    distarr = np.append(distarr,thisdist)
    if (thisdist<mindst):
        mindst=thisdist
        mindstID=i

c = np.c_[rdnm, distarr]  # append result column to array   
c=c[c[:,5].argsort()]     # sort array by score

print('---- TARGET')
print(cfrf) 
print('---- BEST 3 ')
print(c[0:3, 0:6])        # show first 3 lines 
print('---- WORST 3 ')
print(c[(len(rdnm)-3):(len(rdnm)), 0:6])   # show last  3 lines 



---- TARGET
[[14  0 13  3 11]]
---- BEST 3 
[[13.          0.         11.          4.         11.          2.44948974]
 [16.          1.         14.          7.         13.          5.09901951]
 [17.          3.         14.          1.         13.          5.19615242]]
---- WORST 3 
[[ 3.         19.          2.         18.          8.         28.93095228]
 [ 0.         17.          0.         19.         19.         31.20897307]
 [ 0.         18.          0.         18.         19.         31.27299154]]


In [588]:
# TEST 3 - NBA PLAYER DISTANCE 
# DF OF VALUES + TRANSLATED VALUES : FIND CLOSER ARRAYS
# WRONG INPUT DATASET . 

fantasydf = pd.read_excel('NBA2020fantasydf.xls', index_col=0)  
print("file read - NBA2020fantasydf")

data = [
['C',40],
['C-F',32],
['F-C',28],
['F',20],
['F-G',12],
['G-F',8],
['G',0]]
 
MAP_POS=pd.DataFrame(data, columns = ['PLAYER_POSITION','POS_VALUE']) 
fantasydf = pd.merge(fantasydf,MAP_POS,how='left',on='PLAYER_POSITION')

targetplayerID = np.where(fantasydf.PLAYER_NAME=='Trae Young')[0][0]
i = 0
distarr = []
mindst = 100 
mindstID = 0

targetplayer = fantasydf[fantasydf.columns[-15:]].to_numpy()[targetplayerID]

for i, row in fantasydf.iterrows():
    fantarray= row.to_numpy()
    fantarray=fantarray[6:]
    fantasydf.at[i, 'EuclDist']  = distance.euclidean(targetplayer, fantarray)  # euclidean distance

fantasydf.sort_values(by=('EuclDist'), ascending=True).head(10)


file read - NBA2020fantasydf


,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,TEAM_ID,TEAM_ABBREVIATION,GP,MIN,FAN_DUEL_PTS,NBA_FANTASY_PTS,PTS,REB,AST,BLK,STL,TOV,FG3M,FGA,FG_PCT,FTA,FT_PCT,POS_VALUE,EuclDist
56,1629027,Trae Young,G,1610612737,ATL,4,25.7,25.6,27.8,13.8,3.5,5.3,0.50,1.75,4.8,1.5,12.3,0.327,5.3,0.810,0.0,0.000000
60,1628378,Donovan Mitchell,G,1610612762,UTA,3,23.0,24.9,25.6,14.0,3.7,5.0,0.33,0.33,2.3,1.7,12.3,0.351,4.0,0.917,0.0,4.777795
48,1628365,Markelle Fultz,G,1610612753,ORL,4,27.3,26.6,27.9,14.0,4.5,4.5,0.00,1.25,2.0,0.8,13.3,0.434,2.5,0.700,0.0,4.789942
75,203957,Dante Exum,G,1610612739,CLE,2,25.2,23.4,24.9,15.0,2.0,5.0,0.50,1.00,4.5,1.5,9.5,0.526,5.0,0.700,0.0,5.090599
66,1628368,De'Aaron Fox,G,1610612758,SAC,4,26.3,24.0,25.0,12.0,2.0,6.8,0.00,1.00,2.5,1.0,12.8,0.333,4.0,0.625,0.0,5.179456
40,1627749,Dejounte Murray,G,1610612759,SAS,3,25.7,27.6,29.3,11.7,7.3,5.0,0.00,1.67,3.7,0.7,12.7,0.368,2.0,0.833,0.0,6.192625
50,1629632,Coby White,G,1610612741,CHI,4,28.3,26.3,26.8,16.5,4.5,4.3,0.25,0.25,3.0,3.0,13.3,0.453,1.8,0.857,0.0,6.213742
28,202322,John Wall,G,1610612745,HOU,3,22.9,28.9,30.9,16.3,3.0,5.3,0.33,1.67,3.0,1.3,13.3,0.475,4.0,0.583,0.0,6.396775
73,1626156,D'Angelo Russell,G,1610612750,MIN,2,23.4,23.5,24.5,16.5,2.5,3.0,0.00,1.00,2.5,2.5,13.0,0.385,4.0,1.000,0.0,6.594844
74,1628984,Devonte' Graham,G,1610612766,CHA,4,25.9,23.4,24.5,16.0,2.3,4.0,0.00,1.00,3.3,3.3,12.3,0.469,1.5,0.833,0.0,6.671071
